## Overview

This notebook demonstrates how to use a generative AI models for text summarization. It employs [BART](https://arxiv.org/abs/1910.13461), a large language model fine-tuned on the CNN/DailyMail dataset, and [TinyLlama](https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0), a lightweight variant of LLaMA 2, to summarize a [Reuters news article](https://www.reuters.com/world/us/us-pet-adoptions-still-strong-cats-dogs-melt-stress-2021-05-03/) about animal adoption.

## Model Summary

BART is a sequence-to-sequence transformer with a bidirectional (BERT-style) encoder and an autoregressive (GPT-style) decoder. It is pre-trained by corrupting input text and learning to reconstruct the original. While BART excels at text generation tasks like summarization and translation, it also performs well on comprehension tasks such as classification and question answering.

TinyLlama is a compact, 1.1 billion-parameter model based on the LLaMA 2 architecture and tokenizer. Pre-trained on approximately 3 trillion tokens, TinyLlama is designed for efficiency and performs well on fundamental NLP tasks such as text generation and summarization.

## Workflow

1. Install and import required libraries

2. Define a function to generate summaries using BART and TinyLlama

3. Apply the model to a news article on animal adoption


## 1. Install and import required libraries

In [1]:
pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [2]:
!CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 35.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4123171 sha256=ab09c84747a9f218c22276a5e1a589379a9719f073c8120b7b1ee0fad725e92b
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama-cpp-python


In [3]:
!pip3 install huggingface-hub

In [4]:
!huggingface-cli download TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

/usr/local/lib/python3.11/dist-packages/huggingface_hub/commands/download.py:139: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf: 100% 669M/669M [00:01<00:00, 441MB/s]
Download complete. Moving file to tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf
tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf


In [63]:
from transformers import pipeline
from flask import Flask, request, jsonify
from llama_cpp import Llama
from IPython.display import clear_output

## 2. Define a function to generate summaries using BART or TinyLLaMA

In [60]:
# Define a function to summarize text using BART or TinyLLaMA model
def summarize_text(model, text, min, max):
  '''
  model: an integer specifying the model to use for text summarization (1: BART; 2: TinyLLaMA)
  text: a string containing the text to summarize
  min: an integer specifying minimum number of tokens in summarized text
  max: an integer specifying maximum number of tokens in summarized text
  returns: a string containing summarized text
  '''
  if model == 1:
    # Define the summarizer
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

    # Generate output
    summary = summarizer(text, max_length=max, min_length=min, do_sample=False)
    clear_output()
    return summary[0]['summary_text']

  elif model == 2:
    # Define the model
    llm = Llama(model_path="./tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf",
            n_ctx=2048,
            n_threads=8,
            n_gpu_layers=35)

    # Define the prompt
    prompt = "Summarize the following text and limit your answer to around " + str(int((min + max)/2)) + "words (please use complete sentences): " + text

    # Generate output
    response = llm(
        prompt,
        top_p=0.1,      # allow variability in responses
        temperature=.9, # allow exploratory responses
        echo=False,
    )
    clear_output()
    return response['choices'][0]['text']

  else:
    return "Invalid model specified"

## 3. Apply summarize_text function to a news article on animal adoption

#### Use BART model:

In [61]:
news_article = "NEW YORK, May 3 (Reuters) - U.S. pet adoptions are still frolicking as stressed-out families seek warm and fuzzy relief, even with easing lockdowns. Adoption rates at animal shelters jumped as much as 40% in 2020 over the previous year as people coped with isolation at the height of the pandemic. 'There has been such an outpouring from the community for both fostering and adoption since the pandemic,' said Leslie Granger, president and chief executive of Bideawee, a New York nonprofit group which has been finding loving homes for rescued animals since 1903. 'The first week alone last March, we saw more than 700 foster applications come in from families around the New York area,' Granger said. 'We've had an incredible demand for people who want to foster and to adopt for the past year and we're not seeing it slow down. People are still coming in.' At Bideawee's 10,000-square-foot building in Manhattan, about 40 pets are up for adoption. Eight-week-old puppies frolic and a kitten bottle feeds inside the shelter, whose name means 'stay awhile' in Scottish. Bideawee's no-kill policy sets its apart from shelters that put animals to sleep if they are not adopted after a certain period. Even as people open their homes to pets, some COVID-stressed pet owners have dumped unwanted animals on streets. Female cats can have five litters a year, leading to a boom in the stray cat population. Bideawee teaches cat lovers how to trap and neuter feral cats, 'which is the only humane way to reduce the population of community cats,' said Elyise Hallenbeck, Bideawee's Feral Cat Initiative's director of strategy for leadership giving. Enrollment has soared as the courses have moved online. 'Usually pre-pandemic, we would have 30 people in our courses,' she said. 'Nowadays we're getting upwards of 300 from all around the world, including places like Saudi Arabia, Alaska, Brazil, Mexico, Australia.' As Hallenbeck bottle-fed Bodie, a 4-week-old kitten, she said his feral mother passed away soon after giving birth on the streets. 'If you're having a bad moment at work, you can always take a little time out and get some puppy smooches or kitten cuddles,' Granger said."
summarize_text(1, news_article, 100, 200)

"U.S. pet adoptions are still frolicking as stressed-out families seek warm and fuzzy relief. Adoption rates at animal shelters jumped as much as 40% in 2020 over the previous year. Even as people open their homes to pets, some COVID-stressed pet owners have dumped unwanted animals on streets. Bideawee teaches cat lovers how to trap and neuter feral cats, 'which is the only humane way to reduce the population of community cats,' said Elyise Hallenbeck."

Use TinyLLaMA model:

In [62]:
summarize_text(2, news_article, 100, 200)

" 'It's a great way to get out of the house and do something"